In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np

import energypy as ep
import energypylinear as epl

In [3]:
prices = np.random.rand(5) * 100
prices

array([82.69600039, 48.67878125, 11.16553713, 41.04675411, 36.83326004])

In [4]:
linear = epl.Battery(power=2, capacity=4)
opt = linear.optimize(prices)

Optimal


In [5]:
opt

[OrderedDict([('Import [MW]', 0.0),
              ('Export [MW]', 2.2e-11),
              ('Gross [MW]', -2.2e-11),
              ('Net [MW]', -1.9800000000000002e-11),
              ('Losses [MW]', 2.2e-12),
              ('Initial charge [MWh]', 0.0),
              ('Final charge [MWh]', -1.8333333e-12),
              ('Prices [$/MWh]', 82.69600039152019),
              ('Forecast [$/MWh]', 82.69600039152019),
              ('Actual [$/5min]', -1.3644840064600832e-10),
              ('Forecast [$/5min]', -1.3644840064600832e-10)]),
 OrderedDict([('Import [MW]', 1.2e-11),
              ('Export [MW]', -1e-11),
              ('Gross [MW]', 2.2e-11),
              ('Net [MW]', 2.1000000000000002e-11),
              ('Losses [MW]', -1e-12),
              ('Initial charge [MWh]', -1.8333333e-12),
              ('Final charge [MWh]', 0.0),
              ('Prices [$/MWh]', 48.67878125026072),
              ('Forecast [$/MWh]', 48.67878125026072),
              ('Actual [$/5min]', 8.51878671

In [6]:
class FixedAgent:
    def __init__(self, actions):
        self.actions = actions
        self.cursor = -1
        self.memory = []
        
    def act(self, obs):
        if self.cursor == len(self.actions) - 1:
            self.cursor = 0
        else:
            self.cursor += 1
        return self.actions[self.cursor]
    
    def remember(self, *args):
        pass
    
    def reset(self):
        self.cursor = 0

agent = FixedAgent([d['Gross [MW]'] for d in opt])

In [7]:
batt = ep.make_env('battery', power=2, capacity=4, episode_length=len(agent.actions), prices=prices)
batt

<energypy BATTERY env - 2.0 MW 4.0 MWh>

In [8]:
from energypy.experiments.blocks import perform_episode

In [9]:
agent.reset()
step, rewards = perform_episode(agent, batt)

/Users/adam/git/energy-py/energypy/common/spaces/composite.py:110: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if space.data == 'append':


In [14]:
rewards

[-1.5160933405112035e-10,
 -8.113130208376786,
 1.6748305691409409,
 -0.0,
 6.077487906698845e-11]

In [15]:
[d['Actual [$/5min]'] for d in opt]

[-1.3644840064600832e-10,
 8.518786718795627e-11,
 1.8609228546010455,
 -6.157013116786609,
 0.0]

In [16]:
agent.reset()